In [1]:
# Install all libraries:
import sys
!{sys.executable} -m pip install pyjarowinkler --user

import sys
!{sys.executable} -m pip install python-Levenshtein --user

import sys
!{sys.executable} -m pip install Fuzzy --user

import sys
!{sys.executable} -m pip install fuzzywuzzy --user

import sys
!{sys.executable} -m pip install jellyfish --user

You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 720kB/s ta 0:00:01
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /home/oak/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py bdist_wheel for Fuzzy ... done
  Stored in directory: /home/oak/.cache/pip/wheels/79/f7/14/b7e20855729780e85322529469b2d1eadfd940e83d981373cc
Successfully built Fuzzy
You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via

In [2]:
import jellyfish

In [3]:

print( jellyfish.soundex('Omer Ak'))
print( jellyfish.soundex('Omer A'))
print( jellyfish.soundex('Omer'))


O562
O560
O560


In [4]:
import numpy as np
import pyjarowinkler
from pyjarowinkler import distance as jw
import Levenshtein
import fuzzy
from fuzzywuzzy import fuzz
import jellyfish

In [5]:
# From:  https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Longest_common_substring#Python_3
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 1 + len(s1)):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
            else:
                m[x][y] = 0
    return s1[x_longest - longest: x_longest]

In [6]:
#From:  https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Longest_common_subsequence#Computing_the_length_table_of_the_LCS_2
def longest_common_subsequence(X, Y):
    m = len(X)
    n = len(Y)
    # An (m+1) times (n+1) matrix
    C = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
            if X[i-1] == Y[j-1]: 
                C[i][j] = C[i-1][j-1] + 1
            else:
                C[i][j] = max(C[i][j-1], C[i-1][j])
    return C

In [7]:
s1 = 'def abc'
words = s1.split(' ')
s2 = 'Mr. def abc'
words = s2.split(' ')
ss = str()
for w in words:
    ss += w
ss

'Mr.defabc'

In [8]:
n1 = 'def abc'
n2 = 'Mr. def abc'
print (fuzz.ratio(n1,n2)/100.)
print(len(longest_common_substring(n1,n2))/(len(n1)+len(n2))*2.)
print(len(longest_common_subsequence(n1,n2))/(len(n1)+len(n2))*2.)
print(max(max(longest_common_subsequence(n1,n2)))/max(len(n1),len(n2)))
print(max(max(longest_common_subsequence(n1,n2)))/(len(n1)+len(n2))*2.)

0.78
0.7777777777777778
0.8888888888888888
0.6363636363636364
0.7777777777777778


In [9]:
max(max(longest_common_subsequence(n1,n2)))

7

In [14]:
def pp_distance(n1,n2):
    d = np.zeros(8)
    if n1=='' or n2=='': return
    d[0] = jw.get_jaro_distance(n1, n2, winkler=True,scaling=0.1)
    #d[0] = pyjarowinkler.distance.get_jaro_distance(n1, n2, winkler=True,scaling=0.1)
    d[1] = 1.-Levenshtein.distance(n1, n2)/max(len(n1),len(n2))
    d[2] = fuzz.token_sort_ratio(n1, n2)/100.
    d[3] = fuzz.partial_ratio(n1, n2)/100.
    d[4] = len(longest_common_substring(n1,n2))/(len(n1)+len(n2))*2.
    d[5] = max(max(longest_common_subsequence(n1,n2)))/max(len(n1),len(n2))

    d[6] = 1.-Levenshtein.distance(jellyfish.soundex(n1),jellyfish.soundex(n2))/4.
    
    #d[7] = (max(d[0:5]) + np.mean(d[0:5]))/2.
    #d[7] = (max(d[0:5]) * np.mean(d[0:5]))
    #d[8] = X if len(w1)<len(w2)
    # Meta Score:
    d[7] = 1./2.5*(max(d[0],d[1]) + np.mean([d[2],d[3]]) + np.mean([d[4],d[5],d[6]]) - 
                  1./2.*np.min([max(d[0],d[1]), np.mean([d[2],d[3]]), np.mean([d[4],d[5],d[6]])]))
    #d[7] = np.mean( [max(d[0],d[1]) , max(d[2],d[3]) , max(d[4],d[5])] )
    print("{:18}\t{:17}\t{:0.2f}\t{:0.2f}\t{:0.2f}\t{:0.2f}\t{:0.2f}\t{:0.2f}\t{:0.2f}\t{:0.2f}\t{}"
          .format(n1,n2,d[0],d[1],d[2],d[3],d[4],d[5],d[6],d[7],d[7]>=0.70))

In [15]:
print("{:18}\t{:17}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:6}"
      .format("Name1","Name2","JW","Lev","FSort","FPar","sStr","sSeq","Soun","Meta","G'nuf"))

print("• Acceptable Matches:")
pp_distance("Omer Ahmed", "Omer Ahmed")
pp_distance("Omer Ahmed", "Omer D")
pp_distance("Omer Ahmed", "Omar Ahmad")
pp_distance("Omer Ahmed", "Omer")
pp_distance("Omer Ahmed", "Dr Omer Ahmed")
pp_distance("Omer Ahmed", "Dr Omar Ahmad")

print("\n• In Between Matches:")
pp_distance("Omer Ahmed", "Ahmed, Omer")
pp_distance("Omer Ahmed", "Umer")
pp_distance("Omer Ahmed", "Umar")
pp_distance("Omer Ahmed", "Amar Ahmed")
pp_distance("Omer Ahmed", "Dr Ahmed")


print("\n• Not Acceptable Matches:")
pp_distance("Omer Ahmed", "Uncle Om")
pp_distance("Omer Ahmed", "oa")
pp_distance("Omer Ahmed", "Buddy")
pp_distance("Omer Ahmed", "sport guy")

print("\n")
print("{:18}\t{:17}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:6}"
      .format("Name1","Name2","JW","Lev","FSort","FPar","sStr","sSeq","Soun","Meta","G'nuf"))

print("• Acceptable Matches:")
pp_distance("Svetlana Lekoska", "Lana")
pp_distance("Svetlana Lekoska", "Lana")
pp_distance("Svetlana Lekoska", "Svetlana")
pp_distance("Svetlana Lekoska", "S Lekoska")
pp_distance("Svetlana Lekoska", "Lekoska")
pp_distance("Svetlana Lekoska", "Lana Lekoska")
pp_distance("Svetlana Lekoska", "Lekoska, Svetlana")

print("\n• In Between Matches:")
pp_distance("Svetlana Lekoska", "SvetlanaLekoska")
pp_distance("Svetlana Lekoska", "Sveti")


print("\n• Not Acceptable Matches:")

pp_distance("Svetlana Lekoska", "Selna Leksa")
pp_distance("Svetlana Lekoska", "Billy Bob")
pp_distance("Svetlana Lekoska", "Taco Hut")

print("\n")
print("{:18}\t{:17}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:6}"
      .format("Name1","Name2","JW","Lev","FSort","FPar","sStr","sSeq","Soun","Meta","G'nuf"))

print("• Acceptable Matches:")
pp_distance("Yonglin Zhuo", "Yonglin")
pp_distance("Yonglin Zhuo", "Yonglin Z")
pp_distance("Yonglin Zhuo", "Zhuo")

print("\n• In Between Matches:")
pp_distance("Yonglin Zhuo", "Y Z")


print("\n• Not Acceptable Matches:")
pp_distance("Yonglin Zhuo", "Yong Z")
pp_distance("Yonglin Zhuo", "Ying")
pp_distance("Yonglin Zhuo", "Trick")


Name1             	Name2            	JW   	Lev  	FSort	FPar 	sStr 	sSeq 	Soun 	Meta 	G'nuf 
• Acceptable Matches:
Omer Ahmed        	Omer Ahmed       	1.00	1.00	1.00	1.00	1.00	1.00	1.00	1.00	True
Omer Ahmed        	Omer D           	0.87	0.50	0.75	0.83	0.62	0.50	0.75	0.79	True
Omer Ahmed        	Omar Ahmad       	0.86	0.80	0.80	0.80	0.50	0.80	1.00	0.82	True
Omer Ahmed        	Omer             	0.88	0.40	0.57	1.00	0.57	0.40	0.75	0.78	True
Omer Ahmed        	Dr Omer Ahmed    	0.86	0.77	0.87	1.00	0.87	0.77	0.25	0.84	True
Omer Ahmed        	Dr Omar Ahmad    	0.72	0.62	0.70	0.80	0.43	0.62	0.25	0.67	False

• In Between Matches:
Omer Ahmed        	Ahmed, Omer      	0.68	0.36	1.00	0.50	0.48	0.45	0.50	0.67	False
Omer Ahmed        	Umer             	0.68	0.30	0.43	0.75	0.43	0.30	0.50	0.59	False
Omer Ahmed        	Umar             	0.57	0.20	0.29	0.50	0.14	0.20	0.50	0.44	False
Omer Ahmed        	Amar Ahmed       	0.78	0.80	0.80	0.80	0.70	0.80	0.75	0.79	True
Omer Ahmed        	Dr Ahmed         	0.

In [17]:
print("{:18}\t{:17}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:5}\t{:6}"
      .format("Name1","Name2","JW","Lev","FSort","FPar","sStr","sSeq","Soun","Comb","G'nuf"))

pp_distance("Omer Ahmed", "Omer Ahmed")
pp_distance("Omer Ahmed", "Omer Ahmd")
pp_distance("Omer Ahmed", "Omer Ahxx")
pp_distance("Omer Ahmed", "Omer Axxx")
pp_distance("Omer Ahmed", "Omer xxxx")
print()
pp_distance("Omer Ahmed", "Dr Omer Duxx")
pp_distance("Omer Ahmed", "Omerxxumm")
pp_distance("Omer Ahmed", "Omer xxxxxxxxx Duxx")
pp_distance("Omer Ahmed", "OXmXeXr AXhXmXeXd")
pp_distance("Omer Ahmed", "Duxx Omer")
print()
pp_distance("Omer Ahmed", "Omer")
pp_distance("Omer Ahmed", "Ome")
pp_distance("Omer Ahmed", "Om")

Name1             	Name2            	JW   	Lev  	FSort	FPar 	sStr 	sSeq 	Soun 	Comb 	G'nuf 
Omer Ahmed        	Omer Ahmed       	1.00	1.00	1.00	1.00	1.00	1.00	1.00	1.00	True
Omer Ahmed        	Omer Ahmd        	0.98	0.90	0.95	0.89	0.84	0.90	1.00	0.94	True
Omer Ahmed        	Omer Ahxx        	0.90	0.70	0.74	0.78	0.74	0.70	0.75	0.81	True
Omer Ahmed        	Omer Axxx        	0.85	0.60	0.63	0.67	0.63	0.60	0.75	0.73	True
Omer Ahmed        	Omer xxxx        	0.81	0.50	0.42	0.56	0.53	0.50	0.75	0.66	False

Omer Ahmed        	Dr Omer Duxx     	0.59	0.33	0.55	0.53	0.45	0.42	0.25	0.53	False
Omer Ahmed        	Omerxxumm        	0.81	0.50	0.42	0.56	0.42	0.50	0.75	0.64	False
Omer Ahmed        	Omer xxxxxxxxx Duxx	0.75	0.26	0.41	0.50	0.34	0.26	0.75	0.57	False
Omer Ahmed        	OXmXeXr AXhXmXeXd	0.78	0.59	0.74	0.60	0.22	0.59	0.25	0.65	False
Omer Ahmed        	Duxx Omer        	0.54	0.30	0.63	0.44	0.42	0.40	0.25	0.50	False

Omer Ahmed        	Omer             	0.88	0.40	0.57	1.00	0.57	0.40	0.75	0.78	T

In [18]:
5./len("Omer Ahmed")

0.5